# Today's Market

In this notebook, we'll take a look at today's market.

In [20]:
import pandas as pd
from fidap import fidap_client
import config

In [21]:
fidap = fidap_client(api_key=config.FIDAP_API_KEY)

## Aggregate statistics

First, let's get some aggregate statistics - how many companies there are, the total market cap, and the historical average volume.

In [22]:
df = fidap.sql("""SELECT count("ticker") ticker_count, 
       sum("marketcap") marketcap_sum, 
       sum("marketcap") / 1e12 marketcap_sum_tr, 
       sum("mc_ch") mc_ch_sum, 
       sum("mc_ch")/1e9 mc_ch_sum_bn, 
       (sum("mc_ch") / sum("marketcap")) *100 as pct_mc_ch,
       avg("pc_1d") *100 pc_1d_avg,
       stddev("pc_1d") * 100 dispersion,
       sum("revenueusd") as revenue
FROM "FIDAP_DATA_SOURCES"."FIDAP_SCHEMA"."tickers"
WHERE "marketcap" > 100e6;""")
df

,ticker_count,marketcap_sum,marketcap_sum_tr,mc_ch_sum,mc_ch_sum_bn,pct_mc_ch,pc_1d_avg,dispersion,revenue
0,4900,5.011881e+19,50118813.1,138359266900000000,1.383593e+08,0.276063,0.888332,4.795466,4975194415395


In more readable terms - 

In [23]:
print("""
Number of companies: {}
Total market cap: {} trillion
""".format(df.loc[0,'ticker_count'], df.loc[0,'marketcap_sum_tr']))


Number of companies: 4900
Total market cap: 50118813.1 trillion



## Companies with Market Cap > 100,000,000

In [24]:
df_tickers = fidap.sql("""select * from "FIDAP_DATA_SOURCES"."FIDAP_SCHEMA"."tickers" where "marketcap" > 1e8;""")

In [25]:
df_tickers['mktcap_pct_gain'] = (df_tickers['mc_ch']/df_tickers['marketcap'])*100
df_tickers.set_index('date', inplace=True)

### Top companies

We can look at the top companies in several ways - by market cap, revenue, assets, etc. Here are some views - 

#### Top 10 By Market Cap

In [26]:
df_tickers[['ticker','marketcap']].sort_values(by='marketcap', ascending=False).head(10)

,ticker,marketcap
date,,
2021-03-19,AAPL,2.014404e+18
2021-03-19,MSFT,1.737349e+18
2021-03-19,AMZN,1.548441e+18
2021-03-19,GOOGL,1.366448e+18
2021-03-19,FB,8.261375e+17
2021-03-19,BABA,6.504304e+17
2021-03-19,TSLA,6.285793e+17
2021-03-19,BRK.B,5.791954e+17
2021-03-19,JPM,4.734107e+17


#### Top 10 By Revenue

In [27]:
df_tickers[['ticker','revenueusd']].sort_values(by='revenueusd', ascending=False).head(10)

,ticker,revenueusd
date,,
2021-03-19,WMT,1.520790e+11
2021-03-19,AMZN,1.255550e+11
2021-03-19,AAPL,1.114390e+11
2021-03-19,BRK.B,1.037350e+11
2021-03-19,TM,7.874427e+10
2021-03-19,CVS,6.955400e+10
2021-03-19,UNH,6.546700e+10
2021-03-19,MCK,6.259900e+10
2021-03-19,GOOGL,5.689800e+10


#### Top 10 Market Cap Percent Gainers

In [28]:
df_tickers[['ticker','mc_ch', 'marketcap','mktcap_pct_gain']].sort_values(by='mktcap_pct_gain', ascending=False).head(10)

,ticker,mc_ch,marketcap,mktcap_pct_gain
date,,,,
2021-03-19,ARNI,7.500000e+11,5.000000e+11,150.000000
2021-03-19,PFTI,6.666667e+11,1.000000e+12,66.666667
2021-03-19,CLVS,3.922677e+14,8.216000e+14,47.744361
2021-03-19,HOFV,1.549586e+14,3.331000e+14,46.520147
2021-03-19,MDIT,1.022222e+12,2.300000e+12,44.444444
2021-03-19,FHTX,2.172814e+14,5.935000e+14,36.610169
2021-03-19,SNES,3.266071e+12,9.300000e+12,35.119048
2021-03-19,MRKR,4.229778e+13,1.228000e+14,34.444444
2021-03-19,STON,1.027165e+14,3.206000e+14,32.038835


#### Top 10 Marketcap Percent Losers

In [29]:
df_tickers[['ticker','mc_ch', 'marketcap','mktcap_pct_gain']].sort_values(by='mktcap_pct_gain').head(10)

,ticker,mc_ch,marketcap,mktcap_pct_gain
date,,,,
2021-03-19,IDRA,-5.189079e+13,8.370000e+13,-61.996161
2021-03-19,CRGS,-3.600000e+11,6.000000e+11,-60.000000
2021-03-19,KOAN,-3.522581e+12,1.040000e+13,-33.870968
2021-03-19,ESMC,-7.950000e+11,2.400000e+12,-33.125000
2021-03-19,MOJO,-1.131579e+12,4.300000e+12,-26.315789
2021-03-19,GLTC,-1.021277e+12,4.000000e+12,-25.531915
2021-03-19,NUZE,-1.280216e+13,5.990000e+13,-21.372549
2021-03-19,RYMM,-7.789474e+11,3.700000e+12,-21.052632
2021-03-19,SDRLF,-4.820000e+12,2.410000e+13,-20.000000


#### Top 10 Marketcap Gainers

In [30]:
df_tickers[['ticker','mc_ch', 'marketcap','mktcap_pct_gain']].sort_values(by='mc_ch', ascending=False).head(10)

,ticker,mc_ch,marketcap,mktcap_pct_gain
date,,,,
2021-03-19,FB,3.406905e+16,8.261375e+17,4.123896
2021-03-19,AMZN,2.401933e+16,1.548441e+18,1.551194
2021-03-19,BABA,9.243523e+15,6.504304e+17,1.421139
2021-03-19,HD,6.529160e+15,3.112453e+17,2.097754
2021-03-19,WMT,4.938384e+15,3.711210e+17,1.330667
2021-03-19,FDX,4.524085e+15,7.418430e+16,6.098440
2021-03-19,AVGO,4.303103e+15,1.937231e+17,2.221265
2021-03-19,TGT,4.134030e+15,9.351550e+16,4.420689
2021-03-19,BEKE,4.001628e+15,7.578160e+16,5.280474


#### Top 10 Marketcap Losers

In [31]:
df_tickers[['ticker','mc_ch', 'marketcap','mktcap_pct_gain']].sort_values(by='mc_ch').head(10)

,ticker,mc_ch,marketcap,mktcap_pct_gain
date,,,,
2021-03-19,V,-2.518864e+16,4.039335e+17,-6.235838
2021-03-19,MA,-1.012252e+16,3.541436e+17,-2.858311
2021-03-19,AAPL,-9.024956e+15,2.014404e+18,-0.448021
2021-03-19,NKE,-8.599211e+15,2.167516e+17,-3.967312
2021-03-19,JPM,-7.537335e+15,4.734107e+17,-1.592134
2021-03-19,WFC,-4.737193e+15,1.638346e+17,-2.891448
2021-03-19,CB,-3.794889e+15,7.168930e+16,-5.293523
2021-03-19,BAC,-3.502338e+15,3.326367e+17,-1.052902
2021-03-19,BRK.B,-3.372484e+15,5.791954e+17,-0.582270


#### Top Marketcap Gainers by Industry

In [32]:
df_tickers[['mc_ch', 'marketcap','mktcap_pct_gain','industry']]

,mc_ch,marketcap,mktcap_pct_gain,industry
date,,,,
2021-03-19,3.049458e+13,3.738330e+16,0.081573,Diagnostics & Research
2021-03-19,-9.297692e+12,5.799900e+15,-0.160308,Aluminum
2021-03-19,-4.506274e+12,9.337000e+14,-0.482625,Shell Companies
2021-03-19,-2.619802e+12,1.323000e+14,-1.980198,REIT - Mortgage
2021-03-19,1.746004e+14,1.597270e+16,1.093117,Airlines
...,...,...,...,...
2021-03-19,5.023547e+13,1.799900e+15,2.791014,Software - Infrastructure
2021-03-19,1.035189e+13,1.922000e+14,5.385996,Education & Training Services
2021-03-19,1.597055e+14,1.614800e+15,9.890110,Biotechnology


#### Top 10 Industries By Marketcap

In [33]:
df_industries = df_tickers[['industry', 'marketcap','mc_ch']].groupby(['industry']).agg({'marketcap': ['sum','count'], 'mc_ch': ['sum']})
df_industries.columns = ['_'.join(col).strip() for col in df_industries.columns.values]
df_industries['mktcap_pct_gain'] = (df_industries['mc_ch_sum']/df_industries['marketcap_sum'])*100
df_industries.sort_values(by='marketcap_sum', ascending=False).head(10)

,marketcap_sum,marketcap_count,mc_ch_sum,mktcap_pct_gain
industry,,,,
Internet Content & Information,3.031706e+18,50,4.977016e+16,1.641655
Software - Infrastructure,2.908820e+18,82,3.028525e+15,0.104115
Internet Retail,2.829489e+18,30,4.182552e+16,1.478201
Consumer Electronics,2.030149e+18,12,-8.383007e+15,-0.412926
Drug Manufacturers - General,1.833031e+18,13,1.244290e+15,0.067882
Software - Application,1.755848e+18,188,2.373152e+16,1.351570
Semiconductors,1.729821e+18,57,2.159903e+16,1.248628
Credit Services,1.418277e+18,50,-3.423531e+16,-2.413866
Auto Manufacturers,1.265162e+18,17,1.328446e+16,1.050021


#### Top 10 Sectors by Marketcap

In [34]:
df_sectors = df_tickers[['sector', 'marketcap','mc_ch']].groupby(['sector']).agg({'marketcap': ['sum','count'], 'mc_ch': ['sum']})
df_sectors.columns = ['_'.join(col).strip() for col in df_sectors.columns.values]
df_sectors['mktcap_pct_gain'] = (df_sectors['mc_ch_sum']/df_sectors['marketcap_sum'])*100
df_sectors.sort_values(by='marketcap_sum', ascending=False).head(10)

,marketcap_sum,marketcap_count,mc_ch_sum,mktcap_pct_gain
sector,,,,
Technology,1.709023e+19,733,1.438718e+17,0.841836
Financial Services,6.582744e+18,716,-7.798561e+16,-1.184698
Healthcare,6.389536e+18,1020,4.772337e+16,0.746899
Consumer Cyclical,5.759140e+18,542,2.792888e+16,0.484949
Industrials,4.708290e+18,803,-2.468227e+16,-0.524230
Consumer Defensive,3.103733e+18,194,2.071755e+16,0.667504
Real Estate,1.564622e+18,270,-1.233320e+16,-0.788254
Energy,1.539180e+18,240,1.182716e+16,0.768407
Basic Materials,1.452731e+18,249,-2.867490e+15,-0.197386
